In [15]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [16]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2025-01', 'x-cloud-trace-context': '626788e5b017a3d48135d137070be15d', 'date': 'Sun, 16 Mar 2025 21:49:59 GMT', 'server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [17]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Alice Thompson',
  'subject': 'Mathematics',
  'stars': 5,
  'review': 'An engaging professor who makes complex concepts understandable.'},
 {'professor': 'Dr. Benjamin Clark',
  'subject': 'History',
  'stars': 4,
  'review': 'Offers in-depth lectures and thoughtful discussions.'},
 {'professor': 'Dr. Carla Rodriguez',
  'subject': 'Chemistry',
  'stars': 3,
  'review': 'Challenging courses, but her passion for science shines through.'},
 {'professor': 'Prof. David Lee',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Brilliant and approachable, always ready with practical examples.'},
 {'professor': 'Dr. Emily Davis',
  'subject': 'English Literature',
  'stars': 4,
  'review': 'Inspires creativity and encourages critical thinking in every lecture.'},
 {'professor': 'Dr. Frank Martin',
  'subject': 'Physics',
  'stars': 3,
  'review': 'Covers complex topics effectively, though sometimes moves too fast.'},
 {'professor': 'Prof. Grace Kim',
  'subject': 

In [18]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review["professor"],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"],
        }
    })

In [19]:
processed_data[0]

{'values': [-0.017200427129864693,
  -0.025787096470594406,
  -0.05317234620451927,
  0.02429729700088501,
  0.016387809067964554,
  -0.014654221944510937,
  0.003211536444723606,
  0.041849859058856964,
  0.005034849513322115,
  -0.029877277091145515,
  0.05325360596179962,
  0.026383018121123314,
  -0.03713667020201683,
  -0.02605796977877617,
  0.03169212490320206,
  0.006440002471208572,
  -0.028468739241361618,
  -0.008424146100878716,
  0.03632405027747154,
  0.03927656635642052,
  0.028577087447047234,
  -0.017105622217059135,
  0.0309336818754673,
  0.03532182052731514,
  -0.020410271361470222,
  -0.0297689288854599,
  -0.00512288324534893,
  0.0369199737906456,
  0.018988188356161118,
  0.019719544798135757,
  0.06788074225187302,
  -0.012839374132454395,
  -0.02451399527490139,
  -0.05477049574255943,
  -0.011952265165746212,
  0.03223387151956558,
  -0.03315483778715134,
  0.00035086501156911254,
  0.007814682088792324,
  0.03391328454017639,
  0.03063572198152542,
  0.01902

In [20]:
index = pc.Index("rag")
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [21]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20,
 'vector_type': 'dense'}